<a href="https://colab.research.google.com/github/yukaberry/detect_ai_content/blob/feature%2Ffeatures_text/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Step 1: Download the ZIP file using wget
!wget -O model_training_dataset.zip https://storage.googleapis.com/detect-human-ai-generated-raw-data/text_samples/text_samples.zip

# Step 2: Unzip the file
!unzip model_training_dataset.zip


--2024-11-04 10:47:24--  https://storage.googleapis.com/detect-human-ai-generated-raw-data/text_samples/text_samples.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38278427 (37M) [application/zip]
Saving to: ‘model_training_dataset.zip’

model_training_data 100%[===================>]  36.50M  14.8MB/s    in 2.5s    

2024-11-04 10:47:28 (14.8 MB/s) - ‘model_training_dataset.zip’ saved [38278427/38278427]

Archive:  model_training_dataset.zip
   creating: samples/
  inflating: samples/sample_dataset_10000.csv  
  inflating: samples/sample_dataset_1000.csv  
  inflating: samples/sample_dataset_50000.csv  


In [36]:
import pandas as pd

# Corrected file path to include the 'samples/' directory
hybrid_df = pd.read_csv("samples/sample_dataset_1000.csv")
hybrid_df.head()

,Unnamed: 0,text,source,generated
0,667776,"\nEgypt is a land of wonders and fascinations,...",huggingface.co_human_ai_generated_text,1.0
1,969602,These volunteer experiences have taught me inv...,huggingface.co_human_ai_generated_text,1.0
2,38824,Online learning and traditional classroom edu...,NaN,1.0
3,27204,Being a student in college can often be an ove...,NaN,1.0
4,751218,A positive outlook can have contagious effects...,huggingface.co_human_ai_generated_text,1.0


In [38]:
#import gc
#del sample_df  # replace sample_df with any unused variables you want to clear
#gc.collect()

9687

In [37]:

# Remove the ZIP file to save space
!rm model_training_dataset.zip

In [41]:
hybrid_df.shape

(1000, 4)

In [40]:
'''

# Function to process the dataset in chunks
def get_huggingface_texts(number):
    path = "model_training_dataset.csv"
    chunksize = number  # Set chunk size, e.g., 10000 for smaller chunks

    # Initialize empty lists to store chunked data
    human_text_data = []
    ai_text_data = []

    for chunk in pd.read_csv(path, chunksize=chunksize):
        # Process each chunk separately

        # Human text chunk
        human_chunk = chunk[['id', 'human_text', 'instructions']].copy()
        human_chunk.rename(columns={'human_text': 'text'}, inplace=True)
        human_chunk['generated'] = 0  # Label as human-generated
        human_text_data.append(human_chunk)

        # AI text chunk
        ai_chunk = chunk[['id', 'ai_text', 'instructions']].copy()
        ai_chunk.rename(columns={'ai_text': 'text'}, inplace=True)
        ai_chunk['generated'] = 1  # Label as AI-generated
        ai_text_data.append(ai_chunk)

    # Concatenate all chunks after processing
    huggingface_combined_text_df = pd.concat(human_text_data + ai_text_data, ignore_index=True)

    return huggingface_combined_text_df

# Load and process the data in chunks, with each chunk size of 10,000 rows
huggingface_combined_text_df = get_huggingface_texts(10000)
huggingface_combined_text_df.head()

'''

'\n\n# Function to process the dataset in chunks\ndef get_huggingface_texts(number):\n    path = "model_training_dataset.csv"\n    chunksize = number  # Set chunk size, e.g., 10000 for smaller chunks\n\n    # Initialize empty lists to store chunked data\n    human_text_data = []\n    ai_text_data = []\n\n    for chunk in pd.read_csv(path, chunksize=chunksize):\n        # Process each chunk separately\n\n        # Human text chunk\n        human_chunk = chunk[[\'id\', \'human_text\', \'instructions\']].copy()\n        human_chunk.rename(columns={\'human_text\': \'text\'}, inplace=True)\n        human_chunk[\'generated\'] = 0  # Label as human-generated\n        human_text_data.append(human_chunk)\n\n        # AI text chunk\n        ai_chunk = chunk[[\'id\', \'ai_text\', \'instructions\']].copy()\n        ai_chunk.rename(columns={\'ai_text\': \'text\'}, inplace=True)\n        ai_chunk[\'generated\'] = 1  # Label as AI-generated\n        ai_text_data.append(ai_chunk)\n\n    # Concatenate 

In [42]:
# Shuffle the combined dataset after concatenation
s_hybrid_df = hybrid_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [43]:
s_hybrid_df.head()


,Unnamed: 0,text,source,generated
0,373008,Another potential disadvantage is difficulty u...,huggingface.co_human_ai_generated_text,1.0
1,344262,It can actually help create an atmosphere of e...,huggingface.co_human_ai_generated_text,1.0
2,81011,It is important to remember that focusing on w...,huggingface.co_human_ai_generated_text,1.0
3,833496,Whether it's assisting with someone's homework...,huggingface.co_human_ai_generated_text,1.0
4,607382,It has the power to not only reduce the diffic...,huggingface.co_human_ai_generated_text,1.0


In [12]:
!pip install readability-lxml

In [13]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.5 MB/s eta 0:00:00


In [15]:
# Core Libraries
import numpy as np
import pandas as pd

# Text and NLP Libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
import textstat
from textblob import TextBlob
from transformers import pipeline

# Advanced Libraries for Word Embeddings and Semantic Analysis
from sentence_transformers import SentenceTransformer, util

# For Topic Modeling
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# NLTK data downloads
nltk.download('stopwords')
nltk.download('punkt')

# Load Spacy model for NLP
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Text-Based Features
Text-based features provide basic insights into the structure and composition of each text, such as word count, sentence count, and stopword usage.

In [16]:
def text_based_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Word count
    features['word_count'] = text_df[text_column].apply(lambda x: len(word_tokenize(x)))

    # Sentence count
    features['sentence_count'] = text_df[text_column].apply(lambda x: len(sent_tokenize(x)))

    # Average word length
    features['avg_word_length'] = text_df[text_column].apply(lambda x: np.mean([len(word) for word in word_tokenize(x)]))

    # Stopwords count
    features['stopwords_count'] = text_df[text_column].apply(lambda x: sum([1 for word in word_tokenize(x) if word.lower() in stopwords.words('english')]))

    return features

# Lexical Diversity and Readability Scores

Lexical diversity measures the variety of words in text, indicating richness in vocabulary, while readability scores assess how easy a text is to read. Readability metrics such as the Flesch Reading Ease and Dale-Chall Score are used.

In [17]:
def lexical_diversity_readability(text_df, text_column='text'):
    features = pd.DataFrame()

    # Lexical diversity (Unique words / Total words)
    features['lexical_diversity'] = text_df[text_column].apply(lambda x: len(set(word_tokenize(x))) / len(word_tokenize(x)) if len(word_tokenize(x)) > 0 else 0)

    # Readability Scores
    features['flesch_reading_ease'] = text_df[text_column].apply(textstat.flesch_reading_ease)
    features['smog_index'] = text_df[text_column].apply(textstat.smog_index)
    features['flesch_kincaid_grade'] = text_df[text_column].apply(textstat.flesch_kincaid_grade)

    return features

# POS Tagging Features
Part of Speech (POS) tagging helps identify grammatical structures in text, such as noun and verb counts, which can highlight differences in AI and human-generated texts.

In [23]:
def pos_tagging_features(text_df, text_column='text'):
    pos_features = pd.DataFrame()

    def pos_counts(text):
        doc = nlp(text)
        pos_counts = {token.pos_: 0 for token in doc}
        for token in doc:
            pos_counts[token.pos_] += 1
        return pos_counts

    pos_df = text_df[text_column].apply(pos_counts).apply(pd.Series).fillna(0)
    pos_df.columns = [f'pos_{col}' for col in pos_df.columns]
    pos_features = pd.concat([pos_features, pos_df], axis=1)

    return pos_features

# Sentiment and Emotion Analysis

Sentiment and emotion analysis capture the emotional tone of the text, identifying positive, negative, or neutral sentiment and specific emotions.

In [52]:
def sentiment_emotion_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Initialize sentiment analysis pipeline
    sentiment_pipeline = pipeline('sentiment-analysis', truncation=True, max_length=512)# 'pipe : sentiment-analysis part of huggingface Transformers, no model specified default 'mini' BERT

    # Apply sentiment analysis in batches
    texts = text_df[text_column].tolist()
    sentiment_results = sentiment_pipeline(texts, truncation=True, max_length=512, batch_size=32)  # Adjust batch_size as needed

    # Extract sentiment labels from the results
    features['sentiment'] = [result['label'] for result in sentiment_results]

    # Polarity and subjectivity from TextBlob
    features['polarity'] = text_df[text_column].apply(lambda x: TextBlob(x).sentiment.polarity)
    features['subjectivity'] = text_df[text_column].apply(lambda x: TextBlob(x).sentiment.subjectivity)

    return features

# N-grams and Keyword Features
N-grams and keyword features provide insights into common phrases and important keywords, which can distinguish between human and AI-generated texts.

In [25]:
def ngrams_keyword_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Bi-grams count
    vectorizer = CountVectorizer(ngram_range=(2, 2))
    bigram_matrix = vectorizer.fit_transform(text_df[text_column])
    features['bigram_count'] = bigram_matrix.sum(axis=1).A1

    # Trigrams count
    vectorizer = CountVectorizer(ngram_range=(3, 3))
    trigram_matrix = vectorizer.fit_transform(text_df[text_column])
    features['trigram_count'] = trigram_matrix.sum(axis=1).A1

    return features

# Linguistic Complexity Features

These features analyze the complexity of sentences, including the number of clauses and the depth of syntactic structures.

In [26]:
def linguistic_complexity_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Dependency count as a proxy for syntactic complexity
    features['dependency_count'] = text_df[text_column].apply(lambda x: len([token for token in nlp(x).ents]))

    return features

# Semantic Similarity

measures how close the meaning of a text is to a known reference. SBERT embeddings can be used for calculating similarity.

In [27]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_similarity_features(text_df, text_column='text', reference_text="reference text"):
    reference_embedding = model.encode(reference_text)
    features = pd.DataFrame()

    # Cosine similarity with reference text
    features['semantic_similarity'] = text_df[text_column].apply(lambda x: util.cos_sim(model.encode(x), reference_embedding).item())

    return features

# Structural and Formatting Features
Capture line breaks, indentation, or special formatting often present in human-written text.

In [28]:
def structural_formatting_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Line break count
    features['line_break_count'] = text_df[text_column].apply(lambda x: x.count('\n'))

    # Punctuation count
    features['punctuation_count'] = text_df[text_column].apply(lambda x: sum([1 for char in x if char in '.,;!?']))

    return features

# Instruction-Based Features ( given prompt : case of hface df)
 capture specific language or prompts that may be present in instructional texts, like “Task” or “Instructions” keywords.

In [29]:
def instruction_based_features(text_df, text_column='instructions'):
    features = pd.DataFrame()

    # Instruction word count
    features['instruction_word_count'] = text_df[text_column].apply(lambda x: len(word_tokenize(x)) if pd.notna(x) else 0)

    return features

# Word Embedding Features

Embedding features represent text as vectors in a high-dimensional space, capturing nuanced meanings.

In [30]:
def embedding_features(text_df, text_column='text'):
    features = pd.DataFrame()

    # Generate embeddings using SBERT
    embeddings = text_df[text_column].apply(lambda x: model.encode(x))
    embedding_df = pd.DataFrame(embeddings.tolist())
    embedding_df.columns = [f'embedding_{i}' for i in range(embedding_df.shape[1])]
    features = pd.concat([features, embedding_df], axis=1)

    return features

# Topic Modeling Features

Topic modeling identifies latent themes in text, providing insights into the primary subjects discussed.

In [31]:
def topic_modeling_features(text_df, text_column='text', n_topics=5):
    features = pd.DataFrame()
    vectorizer = CountVectorizer(max_df=0.95, min_df=2)
    doc_term_matrix = vectorizer.fit_transform(text_df[text_column])

    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(doc_term_matrix)
    topic_values = lda.transform(doc_term_matrix)
    topic_df = pd.DataFrame(topic_values, columns=[f'topic_{i}' for i in range(n_topics)])
    features = pd.concat([features, topic_df], axis=1)

    return features

In [44]:

# 1. Text-Based Features
text_features = text_based_features(s_hybrid_df, text_column='text')



In [45]:
# 2. Lexical Diversity and Readability Scores
lexical_features = lexical_diversity_readability(s_hybrid_df, text_column='text')

In [46]:
# 3. POS Tagging Features
pos_features = pos_tagging_features(s_hybrid_df, text_column='text')

In [53]:
# 4. Sentiment and Emotion Analysis
sentiment_features = sentiment_emotion_features(s_hybrid_df, text_column='text')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
# 5. N-grams and Keyword Features
ngram_features = ngrams_keyword_features(s_hybrid_df, text_column='text')

In [55]:
# 6. Linguistic Complexity Features
complexity_features = linguistic_complexity_features(s_hybrid_df, text_column='text')

In [56]:
# 7. Semantic Similarity (Advanced)
semantic_features = semantic_similarity_features(s_hybrid_df, text_column='text', reference_text="example reference text")

In [57]:
# 8. Structural and Formatting Features
structural_features = structural_formatting_features(s_hybrid_df, text_column='text')

In [ ]:
# 9. Instruction-Based Features
#instruction_features = instruction_based_features(huggingface_combined_text_df, text_column='instructions')

In [58]:
# 10. Word Embedding Features (Advanced)
embedding_features_df = embedding_features(s_hybrid_df, text_column='text')

In [59]:
# 11. Topic Modeling Features
topic_features = topic_modeling_features(s_hybrid_df, text_column='text', n_topics=5)

In [60]:
# Concatenate all features into a single DataFrame( missinf feauture : instruction_features)
all_features = pd.concat([
    text_features, lexical_features, pos_features, sentiment_features,
    ngram_features, complexity_features, semantic_features,
    structural_features,  embedding_features_df,
    topic_features
], axis=1)

# Display combined features
all_features.head()


,word_count,sentence_count,avg_word_length,stopwords_count,lexical_diversity,flesch_reading_ease,smog_index,flesch_kincaid_grade,pos_DET,pos_ADJ,...,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383,topic_0,topic_1,topic_2,topic_3,topic_4
0,191,9,5.162304,71,0.565445,43.32,14.4,12.0,14.0,19.0,...,0.087723,-0.060280,0.021752,-0.109541,0.031468,0.905390,0.001489,0.001475,0.062669,0.028977
1,266,11,4.864662,112,0.563910,42.41,14.8,12.4,18.0,19.0,...,-0.016650,0.082083,0.015836,-0.127257,0.024186,0.683897,0.113634,0.001077,0.200316,0.001076
2,202,8,4.544554,88,0.544554,48.54,13.4,12.1,12.0,12.0,...,0.017251,0.108373,0.020174,-0.051648,0.013892,0.001422,0.001425,0.001438,0.994300,0.001414
3,228,12,4.206140,96,0.552632,61.77,11.7,9.1,16.0,15.0,...,0.031003,0.016137,0.067800,-0.097144,-0.028914,0.128263,0.383207,0.239736,0.247494,0.001301
4,285,12,4.284211,143,0.547368,60.85,12.8,9.4,29.0,20.0,...,0.064268,0.086684,-0.049943,-0.078347,-0.044467,0.001081,0.001087,0.205162,0.791585,0.001085


In [62]:
all_features.columns.tolist()


['word_count',
 'sentence_count',
 'avg_word_length',
 'stopwords_count',
 'lexical_diversity',
 'flesch_reading_ease',
 'smog_index',
 'flesch_kincaid_grade',
 'pos_DET',
 'pos_ADJ',
 'pos_NOUN',
 'pos_AUX',
 'pos_VERB',
 'pos_PUNCT',
 'pos_ADV',
 'pos_ADP',
 'pos_CCONJ',
 'pos_PRON',
 'pos_SPACE',
 'pos_PART',
 'pos_NUM',
 'pos_SCONJ',
 'pos_PROPN',
 'pos_INTJ',
 'pos_SYM',
 'pos_X',
 'sentiment',
 'polarity',
 'subjectivity',
 'bigram_count',
 'trigram_count',
 'dependency_count',
 'semantic_similarity',
 'line_break_count',
 'punctuation_count',
 'embedding_0',
 'embedding_1',
 'embedding_2',
 'embedding_3',
 'embedding_4',
 'embedding_5',
 'embedding_6',
 'embedding_7',
 'embedding_8',
 'embedding_9',
 'embedding_10',
 'embedding_11',
 'embedding_12',
 'embedding_13',
 'embedding_14',
 'embedding_15',
 'embedding_16',
 'embedding_17',
 'embedding_18',
 'embedding_19',
 'embedding_20',
 'embedding_21',
 'embedding_22',
 'embedding_23',
 'embedding_24',
 'embedding_25',
 'embedding_